# Professor Pattie Maes: AI Agents Pioneer and Fluid Interfaces Expert

### “Traditional AI approaches, which use symbolic knowledge representation that embody fundamental 'rules of thought,' have been turned upside down by the new school, whose adherents write simple small programs that are designed to let intelligence evolve as the programs interact.“ —on the early work of MIT Professor Pattie Maes in Internet Computing 1997, [IEEE (Volume 1, Issue 4)](https://web.archive.org/web/20150924211429/http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=612209&url=http%3A%2F%2Fieeexplore.ieee.org%2Fstamp%2Fstamp.jsp%3Ftp%3D%26arnumber%3D612209)”

![](pattiemaes.jpeg)

### With Prof. Maes we explore the world of agents in 2024 in the context of the MIT Media Lab where she leads research and educational initiatives.



## 🔥 Let's get the required packages

In [ ]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.0-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: YamlDotNet, 13.7.1"

In [ ]:
#!import ../config/Settings.cs
#!import ../config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

## 🥸 We fire up Semantic Kernel's Experimental Agents ...

In [ ]:
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.3.0-alpha"

using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";


In [ ]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

#pragma warning disable SKEXP0101

public class Personay
{
    public string Name { get; set; }
    public string Instructions { get; set; }
    public string Description { get; set; }
}

public class NameGenerator
{
    private List<string> names;
    private int currentIndex = -1;

    public NameGenerator()
    {
        // Initialize the list with ungendered names
        names = new List<string>
        {
            "Alex", "Jordan", "Taylor", "Morgan", "Casey",
            "Riley", "Jamie", "Avery", "Reese", "Skyler",
            "Quinn", "Peyton", "Cameron", "Sawyer", "Drew",
            "Charlie", "Emerson", "Dakota", "Parker", "Sidney"
        };
    }

    public string GetNextName()
    {
        // Increment the index and reset if it exceeds the list count
        currentIndex = (currentIndex + 1) % names.Count;
        return names[currentIndex];
    }
}

 // Track agents for clean-up
static readonly Dictionary<string, IAgent> s_agents = new();

IAgentThread? s_currentThread = null;

async Task<IAgent> CreateAgentAsync(string name, string instructions, string description)
{
    var agent = await new AgentBuilder()
                    .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                    .WithInstructions(instructions)
                    .WithName(name)
                    .WithDescription(description)
                    .BuildAsync();

    return Track(name, agent);
}

async Task CleanUpAsync()
{
    Console.WriteLine("🧽 Cleaning up ...");

    if (s_currentThread != null)
    {
        Console.WriteLine("Thread going away ...");
        s_currentThread.DeleteAsync();
        s_currentThread = null;
    }
    
    if (s_agents.Any())
    {
        Console.WriteLine("Agents going away ...");
        await Task.WhenAll(s_agents.Values.Select(agent => agent.DeleteAsync()));
        s_agents.Clear();
    }
}

IAgent Track(string name, IAgent agent)
{
    s_agents[name] = agent; // Add or update the agent in the dictionary
    return agent;
}



### 🐣 Let's hatch the Media Lab agents

In [ ]:
List<string> personasAvailable = [
    "MediaLabGateway",
    "MediaLabNavigator",
    "MediaLabSpotlight"
];

List<(string Name, string Instructions, string Description)> agentInfo = new();
NameGenerator nameGenerator = new NameGenerator();

var rollcall = "This is the rollcall for the restaurant:\n";

foreach (var (a, i) in personasAvailable.Select((value, idx) => (value, idx + 1)))
{
    var yaml = File.ReadAllText($"../agents/{a}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance) // Use camel case naming convention
        .Build();
    var p = deserializer.Deserialize<Personay>(yaml);
    string fakeName = nameGenerator.GetNextName();
    string desc = $"{p.Name}: {p.Description}";
    string instr = p.Instructions;
    agentInfo.Add((fakeName, instr, desc));
    Console.WriteLine($"Agent defined: {fakeName} --> {desc}");
    rollcall += $"- {i}) {fakeName} is the {desc}\n";
}

foreach (var (name, instructions, description) in agentInfo)
{
    await CreateAgentAsync(name, instructions, description);
}

Console.WriteLine(rollcall);
s_agents

### 🕵️💨 We check which agent will be relevant if a corporate sponsor's arrived

In [ ]:
#pragma warning disable SKEXP0101

IAgentThread? thread = null;

thread = await s_agents["Alex"].NewThreadAsync();
await thread.AddUserMessageAsync(
    $"{rollcall}. A sponsor has arrived at the MIT Media Lab. Who is up?");

var agentMessages = await thread.InvokeAsync(s_agents["Alex"]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

### 🕵️💨 And then we give the agent a whirl

In [ ]:
#pragma warning disable SKEXP0101

var whichAgent = "Alex";

thread = await s_agents[whichAgent].NewThreadAsync();
await thread.AddUserMessageAsync(
    $"Let's pretend a corporate sponsor has arrived at the MIT Media Lab. Please do your thing.");

//Console.WriteLine("🎤 As the corporate sponsor, say something to the agent ...");
//string recognizedText = await speechService.RecognizeOnceAsync();
string recognizedText = "I am a corporate sponsor and I am interested in learning more about the Media Lab.";
await thread.AddUserMessageAsync(recognizedText);

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

//await speechService.SynthesizeSpeechAsync(agentMessages[0].Content);

### 🕵️🗣️ Interact with this agent as you've just arrived

In [ ]:
#pragma warning disable SKEXP0101

//Console.WriteLine("🎤 As the corporate sponsor, say something to the agent ...");
//string recognizedText = await speechService.RecognizeOnceAsync();

string recognizedText = "Thanks. I want to see the work of Pattie Maes. I hear she's a genius.";
await thread.AddUserMessageAsync(recognizedText);

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

//await speechService.SynthesizeSpeechAsync(agentMessages[0].Content);

### 🧐 We'll next want to know which agent is needed next

In [ ]:
#pragma warning disable SKEXP0101

await thread.AddUserMessageAsync(
    $"{rollcall}. Which agent should be used next in this lab visitor scenario?");

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

### 🕵️ You've got a new agent to interact with

In [ ]:
#pragma warning disable SKEXP0101

whichAgent = "Jordan";

await thread.AddUserMessageAsync(
    $"Say something to the guest while introducing yourself.");

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

//await speechService.SynthesizeSpeechAsync(agentMessages[0].Content, "en-US-BrianNeural");

### 🕵️🗣️ Interact with this agent so we can continue down the journey

In [ ]:
#pragma warning disable SKEXP0101

//Console.WriteLine("🎤 As the customer, say something to the agent ...");
//string recognizedText = await speechService.RecognizeOnceAsync();
string recognizedText = "I'm excited to see the work of Pattie Maes. Where is her group located. Are you taking me to see her?";
await thread.AddUserMessageAsync(recognizedText);

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));

//await speechService.SynthesizeSpeechAsync(agentMessages[0].Content, "en-US-BrianNeural");

### 🧐 We'll next want to know which agent is needed next

In [ ]:
#pragma warning disable SKEXP0101

await thread.AddUserMessageAsync(
    $"{rollcall}. Which agent should be used next in this service scenario?");

var agentMessages = await thread.InvokeAsync(s_agents[whichAgent]).ToArrayAsync();

Console.WriteLine(Utils.WordWrap(agentMessages[0].Content, 80));
//await speechService.SynthesizeSpeechAsync(agentMessages[0].Content, "en-US-BrianNeural");